## Import dependencies

In [ ]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import openai
import numpy as np
import pandas as pd
import os
import re
import time
import json

from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
import torch
import os
# disable tokenizers parallelism to avoid warnings or deadlocks
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import warnings
warnings.filterwarnings("ignore")

In [ ]:
default_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(default_credential, "https://cognitiveservices.azure.com/.default" )

api_type = "azure"
api_base = "https://mgh-mind-data-science-private-e2-openai-service.openai.azure.com/" 
api_version = "2024-05-13-preview"

client = openai.AzureOpenAI(api_version=api_version, azure_endpoint=api_base, azure_ad_token_provider=token_provider )

## Staging with progress notes

In [ ]:
note_path = "../cdr_preprocessed_strict_0827.csv"
note_df = pd.read_csv(note_path, index_col=False)
note_df = note_df.drop(columns="Unnamed: 0")
note_df

In [ ]:
notes = list(note_df['CleanedNoteTXT'])
print(notes[0])
print("The global CDR is ",note_df['GlobalCDR'][0])

In [ ]:
def simple_note(prompt, note_text):

    response = client.chat.completions.create(
        model="GPT-4o-model", # model
        messages=[
            {"role": "system", "content": f"{prompt}"},
            {"role": "user", "content": f"{note_text}"}
        ]
    )
    return response.choices[0].message.content

### Attempt 1

In [ ]:
prompt = """Given the progress notes, assess what is the overall level of dementia based on the descriptions of the patient’s cognitive and functional abilities. 
Based on your assessment, classify the severity of the dementia using the Clinical Dementia Rating (CDR) scale, where:
- 0 = No dementia
- 0.5 = Questionable dementia (Very mild impairment)
- 1 = Mild dementia
- 2 = Moderate dementia
- 3 = Severe dementia"""
response = simple_note(prompt, notes[0])
print(response.choices[0].message.content)


### Attempt 2

In [ ]:
prompt = "Given the progress notes, assess what is the overall level of dementia based on the descriptions of the patient’s cognitive and functional abilities, provide the CDR score (0, 0.5, 1, 2, or 3) and a brief justification."
response = simple_note(prompt, notes[0])
print(response.choices[0].message.content)

### Attempt 3 - Return Scores and Save

In [ ]:
prompt = """Given the progress notes, assess what is the overall level of dementia based on the descriptions of the patient’s cognitive and functional abilities, provide the CDR score (0, 0.5, 1, 2, or 3) and a brief justification.
Your response should have this format at the end for final conclusion of CDR score:

**CDR Score:**
[Insert CDR score here]"""
response = simple_note(prompt, notes[0])
print(response.choices[0].message.content)

In [ ]:
def extract_cdr_score(response):
    # Use regex to find the CDR score in the response
    match = re.search(r"\*\*CDR Score:\*\*\s*(\d(\.\d)?)", response)
    if match:
        return match.group(1)
    return None

cdr_score = extract_cdr_score(response)
print("The predicted global CDR is ",cdr_score)
print("The actual global CDR is ",note_df['GlobalCDR'][0])


### Attempt 4 - Add guidance on output format (domains)

In [ ]:
prompt = """Given the progress notes, assess what is the overall level of dementia based on the descriptions of the patient’s cognitive and functional abilities, provide the CDR score (0, 0.5, 1, 2, or 3) and a brief justification.
Your response should follow this format:\n\n
**Summary and Assessment:**\n[Your summary here]\n\n
**Domain-Specific Observations:**\n
1. Memory: [Your observation here]\n
2. Orientation: [Your observation here]\n
3. Judgment and Problem Solving: [Your observation here]\n
4. Community Affairs: [Your observation here]\n
5. Home and Hobbies: [Your observation here]\n
6. Personal Care: [Your observation here]\n\n
**CDR Score:**\n[Insert CDR score here, e.g., 2.0]"""
response = simple_note(prompt, notes[0])
print(response)

In [ ]:
prompt = """Given the progress notes, assess what is the overall level of dementia based on the descriptions of the patient’s cognitive and functional abilities, provide the CDR score (0, 0.5, 1, 2, or 3) and a brief justification. Do not include patient name or any identifiable info in response.
Follow this format:\n\n
**Summary and Assessment:**\n[Your summary here]\n\n
**Domain-Specific Observations:**\n
1. Memory: [Very brief observation here]\n
2. Orientation: [Very brief observation here]\n
3. Judgment and Problem Solving: [Very brief observation here]\n
4. Community Affairs: [Very brief observation here]\n
5. Home and Hobbies: [Very brief observation here]\n
6. Personal Care: [Very brief observation here]\n\n
**CDR Score:**\n[Insert CDR score here, e.g., 2.0]"""
response = simple_note(prompt, notes[0])
print(response)

In [ ]:
prompt = """Given the progress notes, assess what is the overall level of dementia based on the descriptions of the patient’s cognitive and functional abilities, provide the global CDR score (0, 0.5, 1, 2, or 3) and a brief justification. 
Follow this format and do not include patient name in response.:\n\n
**Domain-Specific Observations:**\n
1. Memory: [Very brief observation here]\n
2. Orientation: [Very brief observation here]\n
3. Judgment and Problem Solving: [Very brief observation here]\n
4. Community Affairs: [Very brief observation here]\n
5. Home and Hobbies: [Very brief observation here]\n
6. Personal Care: [Very brief observation here]\n\n
**CDR Score:**\n[Insert barely CDR score here, e.g., 2.0]"""
response = simple_note(prompt, notes[0])
print(response)

In [ ]:
cdr_score = extract_cdr_score(response.choices[0].message.content)
print("The predicted global CDR is ",cdr_score)
print("The actual global CDR is ",note_df['GlobalCDR'][0])

### Attempt 5 - Concise Response

In [ ]:
prompt = """Review the following progress notes and provide a global CDR score. Focus on key observations for each domain (Memory, Orientation, Judgment and Problem Solving, Community Affairs, Home and Hobbies, Personal Care) and give a brief justification for the score. Keep the response concise. Do not include patient name or any identifiable info in response.
Response format:
**CDR Score:** [Insert CDR score here]
**Justification:** [A few sentences summarizing key observations]"""
response = simple_note(prompt, notes[0])
print(response)

In [ ]:
prompt = """Review the following progress notes and provide a global CDR score. Focus on key observations for each domain (Memory, Orientation, Judgment and Problem Solving, Community Affairs, Home and Hobbies, Personal Care).
Keep the response concise and follow this format:\n\n
**CDR Score:** [Insert barely CDR score here, e.g., 2.0. Only assigne if the evidence strongly supports it!]
**Justification:** [A few sentences summarizing key observations. do not include patient name.]"""
response = simple_note(prompt, notes[0])
print(response)

In [ ]:
# Loop through progress notes
results = []
for i, note in enumerate(notes[:10]):
    start_time = time.time() 
    response = simple_note(prompt, note)
    cdr_score = extract_cdr_score(response)
    results.append({
        "CDR Score": cdr_score,
        "Full Response": response
    })

    end_time = time.time()  # End the timer
    duration = end_time - start_time 
    print(f"Time taken for case {i+1}: {duration:.2f} seconds")

In [ ]:
for i, result in enumerate(results):
    print(f"The actual global CDR: {note_df['GlobalCDR'][i]}")
    print(f"The predicted global CDR: {result['CDR Score']}")
    # print("Full Response:\n", result['Full Response'])
    print("\n" + "="*50 + "\n")

In [ ]:
for i, result in enumerate(results):
    print(f"The actual global CDR: {note_df['GlobalCDR'][i]}")
    print(f"The predicted global CDR: {result['CDR Score']}")
    # print("Full Response:\n", result['Full Response'])
    print("\n" + "="*50 + "\n")

### Attempt 6 - Multiple Domains

In [ ]:

def stage_dementia(progress_notes):
    domains = ["Memory", "Orientation", "Judgment and Problem Solving",
               "Community Affairs", "Home and Hobbies", "Personal Care"]
    cdr_scores = {}
    domain_summaries = []
    for domain in domains:
        prompt = f"Analyze the following progress note for information related to {domain}. " \
                 f"Provide classification of the CDR score (0, 0.5, 1, 2, or 3) for the {domain} impairment (be very concise and no need for justification).\n\n" \
        
        domain_response = simple_note(prompt, progress_notes)
        
        # Store the response for this domain
        cdr_scores[domain] = domain_response
        
        # Summarize the response for use in the overall CDR prompt
        domain_summaries.append(f"{domain}: {domain_response}")
    
    # Combine domain summaries into a single text block
    combined_summary = "\n".join(domain_summaries)
    
    # Use the combined summary to inform the overall CDR score
    combination_prompt = "Based on the following classifications for each domain, determine the overall CDR score.\n\n" \
                         f"{combined_summary}\n\n" \
                         "Provide the overall CDR score as a single number (e.g., 'CDR Score: 1.0')."
    overall_cdr = simple_note(combination_prompt, progress_notes)
    
    return overall_cdr, cdr_scores

cdr_score = stage_dementia(notes[0])
print(f"global CDR Score: {cdr_score}")


### Attempt 7 - Less Severe

In [ ]:
prompt = """Review the following progress notes and provide a global CDR score. Focus on key observations for each domain (Memory, Orientation, Judgment and Problem Solving, Community Affairs, Home and Hobbies, Personal Care).
Keep the response concise and follow this format:\n\n
**CDR Score:** [Insert barely CDR score here, e.g., 2.0. Do not assign a high score unless the evidence strongly supports it!]
**Justification:** [A few sentences summarizing key observations. do not include patient name.]"""
response = simple_note(prompt, notes[0])
print(response)

In [ ]:
# Loop through progress notes
results = []
for i, note in enumerate(notes[:10]):
    start_time = time.time() 
    response = simple_note(prompt, note)
    cdr_score = extract_cdr_score(response)
    results.append({
        "CDR Score": cdr_score,
        "Full Response": response
    })

    end_time = time.time()  # End the timer
    duration = end_time - start_time 
    print(f"Time taken for case {i+1}: {duration:.2f} seconds")

In [ ]:
for i, result in enumerate(results):
    print(f"The actual global CDR: {note_df['GlobalCDR'][i]}")
    print(f"The predicted global CDR: {result['CDR Score']}")
    # print("Full Response:\n", result['Full Response'])
    print("\n" + "="*50 + "\n")

In [ ]:
for i, result in enumerate(results):
    print(f"The actual global CDR: {note_df['GlobalCDR'][i]}")
    print(f"The predicted global CDR: {result['CDR Score']}")
    # print("Full Response:\n", result['Full Response'])
    print("\n" + "="*50 + "\n")

### Attempt 8 - Confidence Level

In [ ]:
prompt = """Review the following progress notes and provide a global CDR score. 
Focus on key observations for each domain (Memory, Orientation, Judgment and Problem Solving, Community Affairs, Home and Hobbies, Personal Care). Then, provide a confidence level (low, medium, or high) based on the clarity and consistency of the evidence from these six aspects. If evidence is mixed or not strong, choose "medium" or "low." 
Keep the response concise and follow this format:\n\n
**CDR Score:** [Insert barely CDR score here, e.g., 2.0. Do not assign a high score unless the evidence strongly supports it!]
**Justification:** [A few sentences summarizing key observations. do not include patient name.]
**Confidence Level:** [Insert your confidence level in this decision as "low," "medium," or "high"]"""

response = simple_note(prompt, notes[0])
print(response)

In [ ]:
prompt = """Review the following progress notes and provide a global CDR score.  Do not include patient name in response for information protection.
Focus on key observations for each domain (Memory, Orientation, Judgment and Problem Solving, Community Affairs, Home and Hobbies, Personal Care). Then, provide a confidence level (low, medium, or high) based on the clarity and consistency of the evidence from these six aspects. Use "high" confidence only if the evidence is strong and leaves little doubt about the staging. Otherwise, choose "medium" or "low." 
Keep the response concise and follow this format:\n\n
**CDR Score:** [Insert barely CDR score here, e.g., 2.0. Do not assign a high score unless the evidence strongly supports it!]
**Justification:** [A few sentences summarizing key observations. ]
**Confidence Level:** [Insert your confidence level in this decision as "low," "medium," or "high"]"""

response = simple_note(prompt, notes[0])
print(response)

In [ ]:
def extract_confidence_level(response):
    # Define the regex pattern to match the confidence level
    pattern = r'\*\*Confidence Level:\*\* (low|medium|high)'
    # Search for the pattern in the response
    match = re.search(pattern, response, re.IGNORECASE)
    # Extract and return the confidence level if found
    if match:
        return match.group(1).lower()  # Convert to lowercase for consistency
    else:
        return None

In [ ]:
# Loop through progress notes
results = []
for i, note in enumerate(notes[:10]):
    start_time = time.time() 
    response = simple_note(prompt, note)
    cdr_score = extract_cdr_score(response)
    conf_level = extract_confidence_level(response)
    results.append({
        "CDR Score": cdr_score,
        "Confidence Level": conf_level, 
        "Full Response": response
    })

    end_time = time.time()  # End the timer
    duration = end_time - start_time 
    print(f"Time taken for case {i+1}: {duration:.2f} seconds")

In [ ]:
for i, result in enumerate(results):
    print(f"The actual global CDR: {note_df['GlobalCDR'][i]}")
    print(f"The predicted global CDR: {result['CDR Score']}")
    print(f"The confidence level of prediction is: {result['Confidence Level']}")
    # print("Full Response:\n", result['Full Response'])
    print("\n" + "="*50 + "\n")

### Attempt 9 - Count Domains and Confidence Level

In [ ]:
prompt = """Review the following progress notes and provide a global CDR score.  Do not include patient name in response for information protection.
    Focus on key observations for each domain (Memory, Orientation, Judgment and Problem Solving, Community Affairs, Home and Hobbies, Personal Care) and identify whether there are clear clues that can help determine the dementia stage. Then, summarize how many domains have explicit information mentioned.
    Provide a confidence level (low, medium, or high) based on the clarity and consistency of the evidence from these six domains. Use "high" confidence only if the evidence is explicitly mentioned in most domains and consistent across domains. 
    Keep the response concise and follow this format:\n\n
    **CDR Score:** [Insert barely CDR score here, e.g., 2.0. Do not assign a high score unless the evidence strongly supports it!]
    **Justification:** [A few sentences summarizing key observations. ]
    **# of Domains Explicitly Mentioned:** [Insert number of domains that are clearly observed, e.g., 2]
    **Confidence Level:** [Insert your confidence level in this decision as "low," "medium," or "high"]"""

response = simple_note(prompt, notes[0])
print(response)

In [ ]:
def extract_num_domain(response):
    # Use regex to find the CDR score in the response
    pattern = r'\*\*# of Domains Explicitly Mentioned:\*\* (\d+)'
    match = re.search(pattern, response)
    if match:
        return int(match.group(1))  # Convert to an integer
    else:
        return None  

In [ ]:
# Loop through progress notes
results = []
for i, note in enumerate(notes[:10]):
    start_time = time.time() 
    response = simple_note(prompt, note)
    cdr_score = extract_cdr_score(response)
    num_domain = extract_num_domain(response)
    conf_level = extract_confidence_level(response)
    results.append({
        "CDR Score": cdr_score,
        "Num of Domains": num_domain,
        "Confidence Level": conf_level, 
        "Full Response": response
    })

    end_time = time.time()  # End the timer
    duration = end_time - start_time 
    print(f"Time taken for case {i+1}: {duration:.2f} seconds")

In [ ]:
for i, result in enumerate(results):
    print(f"The actual global CDR: {note_df['GlobalCDR'][i]}")
    print(f"The predicted global CDR: {result['CDR Score']}")
    print(f"Number of explicitly mentioned domains: {result['Num of Domains']}")
    print(f"The confidence level of prediction is: {result['Confidence Level']}")
    # print("Full Response:\n", result['Full Response'])
    print("\n" + "="*50 + "\n")

### Attempt 10 - Make it conservative

In [ ]:
prompt = """Review the following progress notes and provide a global CDR score.  Do not include patient name in response for information protection.
Focus on key observations for each domain (Memory, Orientation, Judgment and Problem Solving, Community Affairs, Home and Hobbies, Personal Care) and identify whether there are clear clues that can help determine the dementia stage. Then, summarize how many domains have explicit information mentioned.
Provide a confidence level (low, medium, or high) based on the clarity and consistency of the evidence from these six domains. Start with an assumption of "low" confidence in your decision. Increase the confidence to "medium" only if there is strong, consistent evidence across multiple domains. Use "high" only if the evidence is really clear and leaves little room for doubt.
Keep the response concise and follow this format:\n\n
**CDR Score:** [Insert barely CDR score here, e.g., 2.0. Do not assign a high score unless the evidence strongly supports it!]
**Justification:** [A few sentences summarizing key observations. ]
**# of Domains Explicitly Mentioned:** [Insert number of domains that are clearly observed, e.g., 2]
**Confidence Level:** [Insert your confidence level in this decision as "low," "medium," or "high"]"""

response = simple_note(prompt, notes[0])
print(response)

In [ ]:
# Loop through progress notes
results = []
for i, note in enumerate(notes[:10]):
    start_time = time.time() 
    response = simple_note(prompt, note)
    cdr_score = extract_cdr_score(response)
    num_domain = extract_num_domain(response)
    conf_level = extract_confidence_level(response)
    results.append({
        "CDR Score": cdr_score,
        "Num of Domains": num_domain,
        "Confidence Level": conf_level, 
        "Full Response": response
    })

    end_time = time.time()  # End the timer
    duration = end_time - start_time 
    print(f"Time taken for case {i+1}: {duration:.2f} seconds")

In [ ]:
for i, result in enumerate(results):
    print(f"The actual global CDR: {note_df['GlobalCDR'][i]}")
    print(f"The predicted global CDR: {result['CDR Score']}")
    print(f"Number of explicitly mentioned domains: {result['Num of Domains']}")
    print(f"The confidence level of prediction is: {result['Confidence Level']}")
    # print("Full Response:\n", result['Full Response'])
    print("\n" + "="*50 + "\n")

### Attempt 11 - Revision

In [ ]:
score_prompt = f"""Based on the following progress notes, classify the patient's global Clinical Dementia Rating (CDR) score into one of the following:
0, 0.5, 1.0, 2.0, or 3.0. Only output the score as a number (e.g., 1.0) with no explanation or formatting.
"""
response = simple_note(score_prompt, notes[0])
print(response)

In [ ]:
def cdr_scoring(prompt, note_text):
    response = client.chat.completions.create(
        model="gpt-4o-model2",
        messages = [
            {"role": "system", "content": "You are a neurologist tasked with cognitive impairment assessment, including Clinical Dementia Rating (CDR) scoring"},
            {"role": "user", 
            "content": f"""Here is the progress note of the patient: {note_text}
            {prompt}
            """}
        ],
        temperature = 0,
        logprobs=True,
        top_logprobs=3
    )
    # Extract top logprobs
    top_logprobs = response.choices[0].logprobs.content[0].top_logprobs
    
    # Format logprobs
    formatted_logprobs = [
        {
            "token": logprob.token,
            "logprob": logprob.logprob,
            "probability_percent": np.round(np.exp(logprob.logprob)*100, 2)
        }
        for logprob in top_logprobs
    ]
    
    return {
        "prediction": response.choices[0].message.content,
        "logprobs": formatted_logprobs
    }

In [ ]:
response = cdr_scoring(score_prompt, notes[0])
print(response)

In [ ]:
question = "What is the patient's ability in Orientation domain?"
probe_prompt = f"""You retrieved the progress note of a patient: {notes[0]}. The question is: {question}.
Before even answering the question, consider whether you have sufficient information in the note to answer the question fully.
Your output should JUST be the boolean true or false, of if you have sufficient information in the note to answer the question.
Respond with just one word, 'True', or the word 'False', nothing else."""

In [ ]:
def domain_sufficiency(prompt, note_text):
    response = client.chat.completions.create(
        model="gpt-4o-model2",
        messages = [
            {"role": "system", "content": "You are a neurologist tasked with cognitive impairment assessment, including Clinical Dementia Rating (CDR) scoring"},
            {"role": "user", 
            "content": f"""Here is the progress note of the patient: {note_text}
            {prompt}
            """}
        ],
        temperature = 0,
        logprobs=True,
        top_logprobs=2
    )
    # Extract top logprobs
    top_logprobs = response.choices[0].logprobs.content[0].top_logprobs
    
    # Format logprobs
    formatted_logprobs = [
        {
            "token": logprob.token,
            "logprob": logprob.logprob,
            "probability_percent": np.round(np.exp(logprob.logprob)*100, 2)
        }
        for logprob in top_logprobs
    ]
    
    return {
        "prediction": response.choices[0].message.content,
        "logprobs": formatted_logprobs
    }

In [ ]:
domain_sufficiency(probe_prompt, notes[0])

In [ ]:
def check_domain_sufficiency(note_text):
    domain_sufficiency = {}
    cdr_domain_questions = {
        "Memory": "What is the patient's ability in CDR Memory domain?",
        "Orientation": "What is the patient's ability in CDR Orientation domain?",
        "Judgment and Problem Solving": "What is the patient's ability in CDR Judgment and Problem Solving domain?",
        "Community Affairs": "What is the patient's ability in CDR Community Affairs domain?",
        "Home and Hobbies": "What is the patient's ability in CDR Home and Hobbies domain?",
        "Personal Care": "What is the patient's ability in CDR Personal Care domain?"
    }
    for domain, question in cdr_domain_questions.items():
        prompt = f"""You retrieved the progress note of a patient: {note_text}. The question is: {question}.
        Before even answering the question, consider whether you have sufficient information in the note to answer the question fully in **formal CDR definitions**.
        Note that you're looking for {domain} domain-specific behaviors or observations.
        Your output should JUST be one word, the boolean 'True' or 'False', of if you have very sufficient information in the note to answer the question.
        """
        response = client.chat.completions.create(
            model="gpt-4o-model2",
            messages = [
                {"role": "system", "content": "You are a neurologist tasked with cognitive impairment assessment, including Clinical Dementia Rating (CDR) scoring"},
                {"role": "user", 
                "content": prompt}
            ],
            temperature = 0,
            logprobs=True,
            top_logprobs=2
        )
        top_logprobs = response.choices[0].logprobs.content[0].top_logprobs
    
        # Format logprobs
        formatted_logprobs = [
            {
                "token": logprob.token,
                "logprob": logprob.logprob,
                "probability_percent": np.round(np.exp(logprob.logprob)*100, 2)
            }
            for logprob in top_logprobs
        ]
        domain_sufficiency[domain] = {
            "answer": response.choices[0].message.content,
            "logprobs": formatted_logprobs
        }

    return domain_sufficiency


In [ ]:
check_domain_sufficiency(notes[0])

## Parallelization and Save Results

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
def simple_note_with_timing(prompt, note_text, id):
    start_time = time.time()
    while True:
        try:
            response = client.chat.completions.create(
                model="GPT-4o-model",
                messages=[
                    {"role": "system", "content": f"{prompt}"},
                    {"role": "user", "content": f"{note_text}"}
                ],
                temperature=0,
                max_tokens=4096
            )
            end_time = time.time()
            duration = end_time - start_time
            return response.choices[0].message.content, duration, id
        except:
            print("Rate limit exceeded. Retrying in 60 seconds...")
            time.sleep(60)

In [ ]:
def divide_into_groups(progress_notes_list, num_groups):
    # Divide the list into 'num_groups' parts
    avg_len = len(progress_notes_list) // num_groups
    groups = [progress_notes_list[i * avg_len:(i + 1) * avg_len] for i in range(num_groups - 1)]
    groups.append(progress_notes_list[(num_groups - 1) * avg_len:])  # Append the remaining items
    return groups, avg_len

# progress_notes_list = note_df
groups, size_of_each_group = divide_into_groups(notes, 43)


In [ ]:
def stage_dementia_parallel(note_text, prompt, max_workers=16):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for index, note in enumerate(note_text):
            futures.append(executor.submit(simple_note_with_timing, prompt, note, index))

        for future in as_completed(futures):
            response, duration, index = future.result()
            cdr = extract_cdr_score(response)
            results.append({"case_id": index, "response": response, "CDR": cdr, "duration": duration})
    # Sort results by case_id to maintain the original order
    results.sort(key=lambda x: x['case_id'])
    return results

In [ ]:
start_group_index = 0
path = "../Results/GPT4o Attempt 4"
prompt = """Given the progress notes, assess what is the overall level of dementia based on the descriptions of the patient’s cognitive and functional abilities, provide the global CDR score (0, 0.5, 1, 2, or 3) and a brief justification. 
        Follow this format and do not include patient name in response.:\n\n
        **Domain-Specific Observations:**\n
        1. Memory: [Very brief observation here]\n
        2. Orientation: [Very brief observation here]\n
        3. Judgment and Problem Solving: [Very brief observation here]\n
        4. Community Affairs: [Very brief observation here]\n
        5. Home and Hobbies: [Very brief observation here]\n
        6. Personal Care: [Very brief observation here]\n\n
        **CDR Score:**\n[Insert barely CDR score here, e.g., 2.0]"""

for group_index, group in enumerate(groups[start_group_index:]):
    print(f"Processing group {group_index + start_group_index + 1} of {len(groups)}...")
    save_results = stage_dementia_parallel(group, prompt, max_workers=16)

    # Step 3: Save the results to a JSON file with the path
    filename = f"group_{group_index + start_group_index + 1}_results.json"
    file_path = os.path.join(path, filename)
    with open(file_path, 'w') as json_file:
        json.dump(save_results, json_file, indent=4)
    print(f"Results saved to {file_path}")
    time.sleep(60)

In [ ]:
start_group_index = 0
path = "../Results/GPT4o Attempt 9"
prompt = """Review the following progress notes and provide a global CDR score.  Do not include patient name in response for information protection.
    Focus on key observations for each domain (Memory, Orientation, Judgment and Problem Solving, Community Affairs, Home and Hobbies, Personal Care) and identify whether there are clear clues that can help determine the dementia stage. Then, summarize how many domains have explicit information mentioned.
    Provide a confidence level (low, medium, or high) based on the clarity and consistency of the evidence from these six domains. Use "high" confidence only if the evidence is explicitly mentioned in most domains and consistent across domains. 
    Keep the response concise and follow this format:\n\n
    **CDR Score:** [Insert barely CDR score here, e.g., 2.0. Do not assign a high score unless the evidence strongly supports it!]
    **Justification:** [A few sentences summarizing key observations. ]
    **# of Domains Explicitly Mentioned:** [Insert number of domains that are clearly observed, e.g., 2]
    **Confidence Level:** [Insert your confidence level in this decision as "low," "medium," or "high"]"""


for group_index, group in enumerate(groups[start_group_index:]):
    print(f"Processing group {group_index + start_group_index + 1} of {len(groups)}...")
    save_results = stage_dementia_parallel(group, prompt, max_workers=16)

    # Step 3: Save the results to a JSON file with the path
    filename = f"group_{group_index + start_group_index + 1}_results.json"
    file_path = os.path.join(path, filename)
    with open(file_path, 'w') as json_file:
        json.dump(save_results, json_file, indent=4)
    print(f"Results saved to {file_path}")
    time.sleep(60)

### Revision

In [ ]:
def cdr_scoring(prompt, note_text, id):
    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-4o-model2",
                messages = [
                    {"role": "system", "content": "You are a neurologist tasked with cognitive impairment assessment, including Clinical Dementia Rating (CDR) scoring"},
                    {"role": "user", 
                    "content": f"""Here is the progress note of the patient: {note_text}
                    {prompt}
                    """}
                ],
                temperature = 0,
                logprobs=True,
                top_logprobs=3,
                max_tokens = 3
            )
            # Extract top logprobs
            top_logprobs = response.choices[0].logprobs.content[0].top_logprobs
            
            # Format logprobs
            formatted_logprobs = [
                {
                    "token": logprob.token,
                    "logprob": logprob.logprob,
                    "probability_percent": np.round(np.exp(logprob.logprob)*100, 2)
                }
                for logprob in top_logprobs
            ]
            
            return {"case_id": id,
                "CDR": response.choices[0].message.content,
                "logprobs": formatted_logprobs}
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Rate limit exceeded. Retrying in 60 seconds...")
            time.sleep(60)

In [ ]:
start_group_index = 0
path = "../Results/GPT4o Attempt 11"
prompt = f"""Based on the following progress notes, classify the patient's global Clinical Dementia Rating (CDR) score into one of the following:
            0, 0.5, 1.0, 2.0, or 3.0. Only output the score as a number (e.g., 1.0) with no explanation or formatting.
            """

if not os.path.exists(path):
    os.makedirs(path)
for group_index, group in enumerate(groups[start_group_index:]):
    print(f"Processing group {group_index + start_group_index + 1} of {len(groups)}...")
    save_results = stage_dementia_parallel(group, prompt, max_workers=16)

    # Step 3: Save the results to a JSON file with the path
    filename = f"group_{group_index + start_group_index + 1}_logprobs.json"
    file_path = os.path.join(path, filename)
    with open(file_path, 'w') as json_file:
        json.dump(save_results, json_file, indent=4)
    print(f"Results saved to {file_path}")
    # time.sleep(60)

In [ ]:
def check_domain_sufficiency(note_text, id):
    while True:
        try:
            domain_sufficiency = {}
            cdr_domain_questions = {
                "Memory": "What is the patient's ability in CDR Memory domain?",
                "Orientation": "What is the patient's ability in CDR Orientation domain?",
                "Judgment and Problem Solving": "What is the patient's ability in CDR Judgment and Problem Solving domain?",
                "Community Affairs": "What is the patient's ability in CDR Community Affairs domain?",
                "Home and Hobbies": "What is the patient's ability in CDR Home and Hobbies domain?",
                "Personal Care": "What is the patient's ability in CDR Personal Care domain?"
            }
            for domain, question in cdr_domain_questions.items():
                prompt = f"""You retrieved the progress note of a patient: {note_text}. The question is: {question}.
                Before even answering the question, consider whether you have sufficient information in the note to answer the question fully in **formal CDR definitions**.
                Note that you're looking for {domain} domain-specific behaviors or observations.
                Your output should JUST be one word, the boolean 'True' or 'False', of if you have very sufficient information in the note to answer the question.
                """
                response = client.chat.completions.create(
                    model="gpt-4o-model2",
                    messages = [
                        {"role": "system", "content": "You are a neurologist tasked with cognitive impairment assessment, including Clinical Dementia Rating (CDR) scoring"},
                        {"role": "user", 
                        "content": prompt}
                    ],
                    temperature = 0,
                    logprobs=True,
                    top_logprobs=2
                )
                top_logprobs = response.choices[0].logprobs.content[0].top_logprobs
            
                # Format logprobs
                formatted_logprobs = [
                    {
                        "token": logprob.token,
                        "logprob": logprob.logprob,
                        "probability_percent": np.round(np.exp(logprob.logprob)*100, 2)
                    }
                    for logprob in top_logprobs
                ]
                domain_sufficiency[domain] = {
                    "answer": response.choices[0].message.content,
                    "logprobs": formatted_logprobs
                }

            return {"case_id": id, "domain_sufficiency": domain_sufficiency}
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Rate limit exceeded. Retrying in 60 seconds...")
            time.sleep(60)


In [ ]:
def check_sufficiency_parallel(note_text, max_workers=16):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for index, note in enumerate(note_text):
            futures.append(executor.submit(check_domain_sufficiency, note, index))

        for future in as_completed(futures):
            response = future.result()
            results.append(response)
    # Sort results by case_id to maintain the original order
    results.sort(key=lambda x: x['case_id'])
    return results

In [ ]:
start_group_index = 0
path = "../Results/GPT4o Attempt 11 - Sufficiency"
if not os.path.exists(path):
    os.makedirs(path)
for group_index, group in enumerate(groups[start_group_index:]):
    group_number = group_index + start_group_index + 1
    filename = f"group_{group_number}_sufficiency.json"
    file_path = os.path.join(path, filename)
    
    if os.path.exists(file_path):
        print(f"File {filename} already exists. Skipping group {group_number}...")
        continue
    else:
        print(f"Processing group {group_number} of {len(groups)}...")
        save_results = check_sufficiency_parallel(group, max_workers=16)
        with open(file_path, 'w') as json_file:
            json.dump(save_results, json_file, indent=4)
        print(f"Results saved to {file_path}")

## RAG model with knowledge base docs

### Load and process data

In [ ]:
with open('../Sources/uds3-ivp-b4.txt', 'r') as file:
    content = file.read()
    paragraphs = content.split('\n\n')
    paragraphs = [paragraph.strip() for paragraph in paragraphs if paragraph.strip()]
    
print(paragraphs)
print(len(paragraphs))

with open('../Sources/uds3-ivp-b7.txt', 'r') as file:
    content = file.read()
    paragraphs2 = content.split('\n\n')
    paragraphs2 = [paragraph.strip() for paragraph in paragraphs2 if paragraph.strip()]
print(paragraphs2)
print(len(paragraphs2))

with open('../Sources/ADL vs. iADL.txt', 'r') as file:
    content = file.read()
    paragraphs3 = content.split('\n\n')
    paragraphs3 = [paragraph.strip() for paragraph in paragraphs3 if paragraph.strip()]
print(paragraphs3)
print(len(paragraphs3))

with open('../Sources/ADL.txt', 'r') as file:
    content = file.read()
    paragraphs4 = content.split('\n\n')
    paragraphs4 = [paragraph.strip() for paragraph in paragraphs4 if paragraph.strip()]
print(paragraphs4)
print(len(paragraphs4))

In [ ]:
documents = paragraphs
documents.extend(paragraphs2)
documents.extend(paragraphs3)
documents.extend(paragraphs4)

# Load the tokenizer for your model (e.g., GPT-4, GPT-3.5-turbo, or any other causal language model)
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Replace with your desired model

# Tokenize the text
tokens = tokenizer.tokenize(documents[12])

# Calculate the number of tokens
num_tokens = len(tokens)

print(f"Number of tokens: {num_tokens}")
print(f"Tokens: {tokens}")


In [ ]:
# define retriever model
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

# encoding documents for retrieval
doc_embeddings = retriever_model.encode(documents, convert_to_tensor=True)


### RAG on single note

In [ ]:
# find most similar docs by comparing embeddings
def retrieve_documents(query, top_k=3): # note text is used as query in this case
    query_embedding = retriever_model.encode(query, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, doc_embeddings)
    top_results = torch.topk(cosine_scores, k=top_k)
    print(top_results[1][0])
    return [documents[idx] for idx in top_results[1][0]]

def augmented_note(prompt, note_text):
    # Retrieve relevant documents
    retrieved_docs = retrieve_documents(note_text)
    additional_context = " ".join(retrieved_docs)
    
    # Combine prompt, note_text, and retrieved documents
    combined_prompt = f"{prompt} Here is some additional information: {additional_context}"
    
    response = client.chat.completions.create(
        model="gpt-4o-model2",
        messages=[
            {"role": "system", "content": combined_prompt},
            {"role": "user", "content": note_text}
        ],
        temperature = 0,
        max_tokens = 4096
    )
    return response.choices[0].message.content

In [ ]:
prompt = """Given the progress notes, assess what is the overall level of dementia based on the descriptions of the patient’s cognitive and functional abilities, provide the global CDR score (0, 0.5, 1, 2, or 3) and a brief justification. 
        Follow this format and do not include patient name in response.:\n\n
        **Domain-Specific Observations:**\n
        1. Memory: [Very brief observation here]\n
        2. Orientation: [Very brief observation here]\n
        3. Judgment and Problem Solving: [Very brief observation here]\n
        4. Community Affairs: [Very brief observation here]\n
        5. Home and Hobbies: [Very brief observation here]\n
        6. Personal Care: [Very brief observation here]\n\n
        **CDR Score:**\n[Insert barely CDR score here, e.g., 2.0]"""
response = augmented_note(prompt, notes[0])

In [ ]:
print(response.choices[0].message.content)

### RAG on multi notes

In [ ]:
def augmented_note_with_timing(prompt, note_text, id):
    start_time = time.time()
    if not isinstance(note_text, str):
        note_text = str(note_text) 
    retrieved_docs = retrieve_documents(note_text)
    additional_context = " ".join(retrieved_docs)
    combined_prompt = f"{prompt} Here is some additional information: {additional_context}"

    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-4o-model2",
                messages=[
                    {"role": "system", "content": combined_prompt},
                    {"role": "user", "content": note_text}
                ],
                temperature = 0,
                max_tokens = 4096
            )
            end_time = time.time()
            duration = end_time - start_time
            return response.choices[0].message.content, duration, id
        except:
            print("Rate limit exceeded. Retrying in 60 seconds...")
            time.sleep(60)

In [ ]:
def divide_into_groups(progress_notes_list, num_groups):
    # Divide the list into 'num_groups' parts
    avg_len = len(progress_notes_list) // num_groups
    groups = [progress_notes_list[i * avg_len:(i + 1) * avg_len] for i in range(num_groups - 1)]
    groups.append(progress_notes_list[(num_groups - 1) * avg_len:])  # Append the remaining items
    return groups, avg_len

progress_notes_list = note_df
groups, size_of_each_group = divide_into_groups(notes, 43)

In [ ]:
def stage_dementia_parallel_rag(note_text, max_workers=16):
    prompt = """Given the progress notes, assess what is the overall level of dementia based on the descriptions of the patient’s cognitive and functional abilities, provide the global CDR score (0, 0.5, 1, 2, or 3) and a brief justification. 
        Follow this format and do not include patient name in response.:\n\n
        **Domain-Specific Observations:**\n
        1. Memory: [Very brief observation here]\n
        2. Orientation: [Very brief observation here]\n
        3. Judgment and Problem Solving: [Very brief observation here]\n
        4. Community Affairs: [Very brief observation here]\n
        5. Home and Hobbies: [Very brief observation here]\n
        6. Personal Care: [Very brief observation here]\n\n
        **CDR Score:**\n[Insert barely CDR score here, e.g., 2.0]"""
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for index, note in enumerate(note_text):
            futures.append(executor.submit(augmented_note_with_timing, prompt, note, index))

        for future in as_completed(futures):
            response, duration, index = future.result()
            cdr = extract_cdr_score(response)
            results.append({"case_id": index, "response": response, "CDR": cdr, "duration": duration})
    # Sort results by case_id to maintain the original order
    results.sort(key=lambda x: x['case_id'])
    return results

In [ ]:
start_group_index = 0
path = "../Results/RAG GPT"

for group_index, group in enumerate(groups[start_group_index:]):
    print(f"Processing group {group_index + start_group_index + 1} of {len(groups)}...")
    save_results = stage_dementia_parallel_rag(group, max_workers=16)

    # Step 3: Save the results to a JSON file with the path
    filename = f"group_{group_index + start_group_index + 1}_results.json"
    file_path = os.path.join(path, filename)
    with open(file_path, 'w') as json_file:
        json.dump(save_results, json_file, indent=4)
    print(f"Results saved to {file_path}")
    time.sleep(60)